## Process Data

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')
%matplotlib inline

In [48]:
df = pd.read_csv("./data/data/data_cvd.csv", index_col=0, decimal='.')
df = df.convert_dtypes()
# df.info()

In [49]:
# drop features
target = df["CVD Death"]
df_feat = df.iloc[:, :83]
df_feat = pd.concat([df_feat.iloc[:, :30], df_feat.iloc[:, 40:]], axis=1)
df_feat = df_feat.drop(columns=["Patologia TIROIDE ", "Controllo", "AMI", "PMI", "Weight", "Height"])
df_feat["CVD Death"] = target
df_feat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4762 entries, 8986 to 1193
Data columns (total 68 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Age (1)                                     4762 non-null   Int64  
 1   Gender (Male = 1)                           4762 non-null   Int64  
 2   Angina                                      4762 non-null   Int64  
 3   Previous CABG                               4762 non-null   Int64  
 4   Previous PCI                                4762 non-null   Int64  
 5   Previous 
Myocardial Infarction             4762 non-null   Int64  
 6   Acute 
Myocardial Infarction                4762 non-null   Int64  
 7   Angiography                                 4762 non-null   Int64  
 8   Vessels                                     3041 non-null   Float64
 9   CAD                                         4762 non-null   Int64  
 10  Smoke
His

In [50]:
# df_feat = df_feat.dropna()
print("Total patient: \t", len(df_feat))
print("Survived: \t", len(df_feat)-df_feat["CVD Death"].sum(), "\t", (len(df_feat)-df_feat["CVD Death"].sum())/len(df_feat))
print("Dead: \t\t", df_feat["CVD Death"].sum(), "\t", df_feat["CVD Death"].sum()/len(df_feat))

Total patient: 	 4762
Survived: 	 4351 	 0.9136917261654767
Dead: 		 411 	 0.08630827383452332


In [51]:
# dataset split
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df_feat, test_size=0.2, stratify=df_feat.iloc[:,-1])
df_train, df_valid = train_test_split(df_train, test_size=0.2, stratify=df_train.iloc[:,-1])
df_train.to_csv(f"data/data/train.csv")
df_valid.to_csv(f"data/data/valid.csv")
df_test.to_csv(f"data/data/test.csv")
print("Train: \t", len(df_train))
print("Valid: \t", len(df_valid))
print("Test: \t", len(df_test))

Train: 	 3047
Valid: 	 762
Test: 	 953


## Two Datasets
Created two different dataset with the same train, valid, test split for fair comparison to assess the influence of vessel feature.
1. no_vessels: removed completely the vessels column
2. with_vessels: with the vessels feature, need to apply the dropna to drop patients without vessel

In [67]:
# df_t = pd.read_csv("./data/data/test.csv", index_col=0, decimal='.')
# df_t = df_t.convert_dtypes()
# df_t = df_t.drop(columns=["Vessels"])
# df_t = df_t.dropna()
# df_t.info()

# df_t.to_csv("data/data/no_vessels/test.csv")

In [66]:
# import seaborn as sns
# fig, ax = plt.subplots(figsize=(35,30))
# sns.heatmap(df_feat.corr(), annot=True)
# fig.savefig("feat.pdf")